In [16]:
import sqlite3
import pandas as pd
import re

## IC Violation Check for Address

In [ ]:
ori_data = pd.read_csv('./data/farmersmarkets-2017-01-10.csv')
clean_data = pd.read_csv('./data/address.csv')

In [5]:
conn_ori = sqlite3.connect("farmersmarkets.db") 
ori_data.to_sql('farmersmarkets', conn_ori)
conn_clean = sqlite3.connect("cleanfarmersmarkets.db") 
clean_data.to_sql('cleanfarmersmarkets', conn_clean)

In [6]:
# city, state and county are not empty
empty_address = '''
SELECT FMID, MarketName, city, state, county 
FROM farmersmarkets 
WHERE city IS NULL
OR state IS NULL
OR county IS NULL;
'''
data = pd.read_sql(empty_address, conn_ori)
data

,FMID,MarketName,city,State,County
0,1009364,106 S. Main Street Farmers Market,Six Mile,South Carolina,None
1,1006234,4th Street Farmers Market,Larimer,Colorado,None
2,1006494,52 & Shadeland Avenue Farmers Market,Indianapolis,Indiana,None
3,1009543,ABV Farm Market,Mount Bethe,Pennsylvania,None
4,1001300,Algonquin Farmers Market,Algonquin,Illinois,None
...,...,...,...,...,...
546,1009255,Wood Dale Farmer's Market,Wood Dale,Illinois,None
547,1007624,Worcester Northeast Side Farmers Market,Worcester,Massachusetts,None
548,1009336,Yazoo Farmers Market,Yazoo City,Mississippi,None
549,1009531,Year-Round Cedar City Farmer's Market,Cedar City,Utah,None


In [7]:
empty_address = '''
SELECT FMID, MarketName, city, state, county 
FROM cleanfarmersmarkets 
WHERE city IS NULL
OR state IS NULL
OR county IS NULL;
'''
data = pd.read_sql(empty_address, conn_clean)
data

,FMID,MarketName,City,State,County


In [8]:
same_address = '''
SELECT *
FROM 
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM farmersmarkets
) AS m1,
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM farmersmarkets
) AS m2 
WHERE  m1.zip = m2.zip 
AND m1.county = m2.county 
AND m1.city = m2.city 
AND m1.street = m2.street 
AND m1.state = m2.state 
AND m1.fmid <> m2.fmid; 
'''
data = pd.read_sql(same_address, conn_ori)
data

,FMID,MarketName,zip,street,city,county,state,FMID,MarketName,zip,street,city,county,state
0,1009719,CIty of Momence,60954,123 W. River St.,Momence,Kankakee,Illinois,1001115,Momence Farmer's Market,60954,123 W. River St.,Momence,Kankakee,Illinois
1,1000865,Columbus Winter Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio,1008799,Edible Thursdays Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio
2,1008799,Edible Thursdays Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio,1000865,Columbus Winter Farmers Market,43214,4300 Indianola Ave.,Columbus,Franklin,Ohio
3,1004024,Farmer's Market @ Woburn's Spence Farm,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts,1006893,Woburn Farmers Market,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts
4,1004024,Farmer's Market @ Woburn's Spence Farm,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts,1008931,Woburn Indoor Farmers Market,01801,41 Wyman Street,Woburn,Middlesex,Massachusetts
5,1000396,Four Seasons Market - Richardson,75080,677 W. Campbell Rd,Richardson,Dallas,Texas,1009996,Richardson Farmers Market,75080,677 W. Campbell Rd,Richardson,Dallas,Texas
6,1007285,Hamilton Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama,1008856,Marion County Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama
7,1011624,Main Street Farmers & Artists Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland,1005103,Main Street Farmers Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland
8,1005103,Main Street Farmers Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland,1011624,Main Street Farmers & Artists Market,20878,301 Main Street,Gaithersburg,Montgomery,Maryland
9,1008856,Marion County Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama,1007285,Hamilton Farmers Market,35570,2151 Bexar Avenue West,Hamilton,Marion,Alabama


In [9]:
same_address = '''
SELECT *
FROM 
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM cleanfarmersmarkets
) AS m1,
(
SELECT FMID, MarketName, zip, street, city, county, state 
FROM cleanfarmersmarkets
) AS m2 
WHERE  m1.zip = m2.zip 
AND m1.county = m2.county 
AND m1.city = m2.city 
AND m1.street = m2.street 
AND m1.state = m2.state 
AND m1.fmid <> m2.fmid; 
'''
data = pd.read_sql(same_address, conn_clean)
data

,FMID,MarketName,zip,street,city,county,state,FMID,MarketName,zip,street,city,county,state


## IC Violation Check for Season

In [5]:
data = pd.read_csv('./data/farmersmarkets.csv', low_memory=False)

In [6]:
seasons = data[['FMID', 'Season1Date 1', 'Season2Date 1', 'Season3Date 1', 'Season4Date 1']]
seasons

,FMID,Season1Date 1,Season2Date 1,Season3Date 1,Season4Date 1
0,1012063,06/08/2016 to 10/12/2016,NaN,NaN,NaN
1,1011871,06/25/2016 to 10/01/2016,NaN,NaN,NaN
2,1011878,05/04/2016 to 10/12/2016,NaN,NaN,NaN
3,1009364,NaN,NaN,NaN,NaN
4,1010691,04/02/2014 to 11/30/2014,NaN,NaN,NaN
...,...,...,...,...,...
8660,1004767,07/04/2014 to 10/24/2014,NaN,NaN,NaN
8661,1000778,06/07/2016 to 10/04/2016,NaN,NaN,NaN
8662,1012380,05/07/2016 to 10/15/2016,NaN,NaN,NaN
8663,1004686,NaN,NaN,NaN,NaN


In [8]:
seasons = seasons.melt(
    id_vars=["FMID"], 
    var_name="SeasonNum",
    value_name="SeasonDate"
)[['FMID', 'SeasonDate']].dropna().reset_index(drop=True)

seasons

,FMID,SeasonDate
0,1012063,06/08/2016 to 10/12/2016
1,1011871,06/25/2016 to 10/01/2016
2,1011878,05/04/2016 to 10/12/2016
3,1010691,04/02/2014 to 11/30/2014
4,1002454,July to November
...,...,...
5896,1012508,01/15/2017 to 01/15/2017
5897,1005991,05/01/2015 to
5898,1010118,04/14/2016 to 04/14/2016
5899,1001875,05/28/2016 to 09/24/2016


In [11]:
seasons['SeasonDate'] = [s.strip() for s in seasons['SeasonDate']]
seasons['SeasonDateList'] = [[d for d in s if d] for s in seasons['SeasonDate'].str.split(' to')]
seasons['SeasonStart'] = [s[0] for s in seasons['SeasonDateList']]
seasons['SeasonEnd'] = [s[1] if len(s) > 1 else s[0] for s in seasons['SeasonDateList']]
seasons['MostRecentOpeningDate'] = pd.to_datetime(seasons['SeasonEnd'], errors = 'coerce')

seasons

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
0,1012063,06/08/2016 to 10/12/2016,"[06/08/2016, 10/12/2016]",06/08/2016,10/12/2016,2016-10-12
1,1011871,06/25/2016 to 10/01/2016,"[06/25/2016, 10/01/2016]",06/25/2016,10/01/2016,2016-10-01
2,1011878,05/04/2016 to 10/12/2016,"[05/04/2016, 10/12/2016]",05/04/2016,10/12/2016,2016-10-12
3,1010691,04/02/2014 to 11/30/2014,"[04/02/2014, 11/30/2014]",04/02/2014,11/30/2014,2014-11-30
4,1002454,July to November,"[July, November]",July,November,NaT
...,...,...,...,...,...,...
5896,1012508,01/15/2017 to 01/15/2017,"[01/15/2017, 01/15/2017]",01/15/2017,01/15/2017,2017-01-15
5897,1005991,05/01/2015 to,[05/01/2015],05/01/2015,05/01/2015,2015-05-01
5898,1010118,04/14/2016 to 04/14/2016,"[04/14/2016, 04/14/2016]",04/14/2016,04/14/2016,2016-04-14
5899,1001875,05/28/2016 to 09/24/2016,"[05/28/2016, 09/24/2016]",05/28/2016,09/24/2016,2016-09-24


### Validation for Special Cases

In [12]:
seasons.loc[seasons['FMID'] == 1000165, ]

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
4452,1000165,March to November,"[March, November]",March,November,NaT


In [13]:
seasons.loc[seasons['FMID'] == 1000788, ]

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
5135,1000788,"July 9, 2012 to October 29, 2012","[July 9, 2012, October 29, 2012]","July 9, 2012","October 29, 2012",2012-10-29


In [14]:
seasons.loc[seasons['FMID'] == 1000961, ] 

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
5114,1000961,05/01/2015 to,[05/01/2015],05/01/2015,05/01/2015,2015-05-01


In [15]:
seasons.loc[seasons['FMID'] == 1001139, ] 

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
2196,1001139,"April to Sept 24, 2011","[April, Sept 24, 2011]",April,"Sept 24, 2011",2011-09-24


### Before & After Comparison - IC Violation Check

In [ ]:
seasons.loc[[',' in s for s in seasons.SeasonEnd],]

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
1476,1000107,"May 5, 2012 to Oct 6, 2012","[May 5, 2012, Oct 6, 2012]","May 5, 2012","Oct 6, 2012",2012-10-06
5135,1000788,"July 9, 2012 to October 29, 2012","[July 9, 2012, October 29, 2012]","July 9, 2012","October 29, 2012",2012-10-29
5134,1000789,"July 12, 2012 to October 25, 2012","[July 12, 2012, October 25, 2012]","July 12, 2012","October 25, 2012",2012-10-25
2198,1001137,"April to November 4, 2011","[April, November 4, 2011]",April,"November 4, 2011",2011-11-04
2196,1001139,"April to Sept 24, 2011","[April, Sept 24, 2011]",April,"Sept 24, 2011",2011-09-24
...,...,...,...,...,...,...
4306,1005772,"May 7, 2011 to October 15, 2011","[May 7, 2011, October 15, 2011]","May 7, 2011","October 15, 2011",2011-10-15
2130,1005993,"May 25, 2012 to August 31, 2012","[May 25, 2012, August 31, 2012]","May 25, 2012","August 31, 2012",2012-08-31
1513,1006135,"August 2, 2012 to September 27, 2012","[August 2, 2012, September 27, 2012]","August 2, 2012","September 27, 2012",2012-09-27
1849,1006688,"June 30, 2012 to September 1 , 2012","[June 30, 2012, September 1 , 2012]","June 30, 2012","September 1 , 2012",2012-09-01


In [ ]:
seasons.loc[[',' in str(s) for s in seasons.MostRecentOpeningDate],]

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate


In [ ]:
seasons.loc[['/' in s for s in seasons.SeasonEnd],]

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
1397,1000003,05/31/2014 to 10/31/2014,"[05/31/2014, 10/31/2014]",05/31/2014,10/31/2014,2014-10-31
5642,1000008,05/21/2016 to 10/29/2016,"[05/21/2016, 10/29/2016]",05/21/2016,10/29/2016,2016-10-29
3123,1000008,05/23/2015 to 11/01/2015,"[05/23/2015, 11/01/2015]",05/23/2015,11/01/2015,2015-11-01
5865,1000008,11/05/2016 to 04/29/2017,"[11/05/2016, 04/29/2017]",11/05/2016,04/29/2017,2017-04-29
4390,1000009,06/14/2014 to 10/11/2014,"[06/14/2014, 10/11/2014]",06/14/2014,10/11/2014,2014-10-11
...,...,...,...,...,...,...
2266,1012844,05/06/2017 to 10/28/2017,"[05/06/2017, 10/28/2017]",05/06/2017,10/28/2017,2017-10-28
3356,1016768,05/07/2016 to 10/01/2016,"[05/07/2016, 10/01/2016]",05/07/2016,10/01/2016,2016-10-01
2539,1016770,10/02/2016 to 02/04/2017,"[10/02/2016, 02/04/2017]",10/02/2016,02/04/2017,2017-02-04
1577,2000005,01/01/2014 to 12/31/2014,"[01/01/2014, 12/31/2014]",01/01/2014,12/31/2014,2014-12-31


In [ ]:
seasons.loc[['/' in str(s) for s in seasons.MostRecentOpeningDate],]

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate


In [ ]:
seasons.loc[[re.search(r'\d', s) is None for s in seasons.SeasonEnd],]

,FMID,SeasonDate,SeasonDateList,SeasonStart,SeasonEnd,MostRecentOpeningDate
3643,1000010,April to October,"[April, October]",April,October,NaT
1860,1000011,April to October,"[April, October]",April,October,NaT
4629,1000016,May to October,"[May, October]",May,October,NaT
1447,1000022,January to December,"[January, December]",January,December,NaT
483,1000023,January to December,"[January, December]",January,December,NaT
...,...,...,...,...,...,...
769,1007672,May to October,"[May, October]",May,October,NaT
601,1007741,April to October,"[April, October]",April,October,NaT
834,1007752,May to November,"[May, November]",May,November,NaT
4836,1008037,August to August,"[August, August]",August,August,NaT


In [ ]:
seasons.loc[[re.search(r'\d', s) is None for s in seasons.SeasonEnd], 'MostRecentOpeningDate'].unique()

array(['NaT'], dtype='datetime64[ns]')